<a id='top'></a>

# Build Dataset for Faults Analysis against Weather Data
Munge datasets from multiple sources into a single dataframe for analysis
<br>
Step 1: [Setup file and folder paths](#setup)<br>
Step 2: [Build dataframe from multiple files](#multiple)<br>
  or  : [Load previously build dataframe](#single)<br>
Step 3: [Clean dataframe](#clean)<br>
Step 4: [Combine into single dataframe](#combine)<br>
Step 5: [Write dataframe to file](#write)

<a id='setup'></a>

## Setup file and folder paths

In [8]:
import os
from os import path
import pandas as pd

current_folder = os.getcwd()
data_folder = path.join(path.abspath('..'), 'data')
bom_data_folder = path.join(path.abspath('..'), 'data\BOM')
output_folder = path.join(path.abspath('..'), 'output')

print ('Current working folder: ' + current_folder)
print ('Current data folder: ' + data_folder)
print ('Current output folder: ' + output_folder)
print ('BOM data folder: ' + bom_data_folder)
print ('Files in data folder:' + str(os.listdir(data_folder)))

Current working folder: C:\Users\tdavies\Desktop\Data_Science\Projects\VV_Forecasting\notebooks
Current data folder: C:\Users\tdavies\Desktop\Data_Science\Projects\VV_Forecasting\data
Current output folder: C:\Users\tdavies\Desktop\Data_Science\Projects\VV_Forecasting\output
BOM data folder: C:\Users\tdavies\Desktop\Data_Science\Projects\VV_Forecasting\data\BOM
Files in data folder:['BOM', 'BOM Station Locations.xlsx', 'corrective_maint_against_weather.csv', 'corrective_maint_job_types.csv', 'DS FLOC.xlsx', 'FLOC (minus DS).xlsx', 'MiningProcess_Flotation_Plant_Database.csv', 'Notifs 2009-2018.xlsx', 'WA_Postcodes.xlsx', 'Weather.xlsx']


<a id='single'></a>

## Load dataframe from previously built xlsx

In [5]:
filename = 'corrective_maint_against_weather.csv'
#filename = 'binary_weather.csv'
df_complete = pd.read_csv(path.join(data_folder, filename))
print ('File loaded.')

File loaded.


In [6]:
df_complete.head()

,Unnamed: 0,notif_date,floc,order,job_type,object_part_code,object_part_text,damage_code,damage_code_text,cause_code,...,weather_date,evapo_trans_0000_2400,rain_0900_0900,pan_evap_0900_0900,max_temp,min_temp,max_rel_humidity,min_rel_humidity,avg_10m_wind_speed,solar_radiation
0,0,2011-12-21,504107.0,NaN,SPH,NaN,NaN,NaN,NaN,NaN,...,2011-12-21,7.3,0.0,NaN,28.9,13.6,97.0,32.0,3.87,34.47
1,1,2011-12-21,673096.0,NaN,SPL,NaN,NaN,NaN,NaN,NaN,...,2011-12-21,7.9,0.0,NaN,30.6,16.4,82.0,35.0,4.24,34.47
2,2,2011-12-21,810841.0,NaN,SPH,NaN,NaN,NaN,NaN,NaN,...,2011-12-21,NaN,NaN,NaN,25.8,NaN,91.0,61.0,4.63,34.24
3,3,2011-12-21,27.0,NaN,SPH,NaN,NaN,NaN,NaN,NaN,...,2011-12-21,7.2,0.0,NaN,29.2,17.4,89.0,38.0,3.37,34.38
4,4,2011-12-21,117547.0,NaN,SF1,PIPE,Pipe,BROK,Broken,NaN,...,2011-12-21,7.3,0.0,NaN,28.9,13.6,97.0,32.0,3.87,34.47


<a id='multiple'></a>

## Build single weather dataframe from multiple files

In [9]:
# Import multiple CSV files from BOM into one dataframe
# Takes about a minute to run...
import glob

all_stations = pd.DataFrame()

folder_list = os.listdir(bom_data_folder)
for folder in folder_list:
    csv_list = glob.glob(path.join(path.abspath(bom_data_folder), folder+'\*.csv'))
    df = pd.concat([pd.read_csv(f, header=None, skiprows=13) for f in csv_list], ignore_index=True)
    all_stations = all_stations.append(df , ignore_index=False)

# Add dataframe column headings
all_stations.columns=['station_name', 'weather_date', 'evapo_trans_0000_2400', 'rain_0900_0900', 'pan_evap_0900_0900',
                          'max_temp','min_temp', 'max_rel_humidity','min_rel_humidity', 'avg_10m_wind_speed','solar_radiation']

# Remove totals rows and reindex
all_stations = all_stations[all_stations['station_name'] != 'Totals:']
all_stations = all_stations.reset_index(drop=True)

In [3]:
#all_stations

## Load raw data into individual dataframes

In [10]:
# Load data into individual dataframes
# Takes about two minutes to load...

df_notifs = pd.ExcelFile(path.join(data_folder, 'Notifs 2009-2018.xlsx')).parse('Sheet1')
print ('Maintenance data loaded.')
df_ds_floc_master_data = pd.ExcelFile(path.join(data_folder, 'DS FLOC.xlsx')).parse('DS FLOC')
print ('Domestic service assets loaded.')
df_else_floc_master_data = pd.ExcelFile(path.join(data_folder, 'FLOC (minus DS).xlsx')).parse('FL')
print ('Other assets loaded.')
df_postcodes = pd.ExcelFile(path.join(data_folder, 'WA_Postcodes.xlsx')).parse('postcodes')
print ('Post codes loaded.')
df_stations = pd.ExcelFile(path.join(data_folder, 'BOM Station Locations.xlsx')).parse('Stations')
print ('Weather stations loaded.')
#df_weather = pd.ExcelFile(path.join(data_folder, 'Weather.xlsx')).parse('Sheet1')
print ('Weather data loaded.')

Maintenance data loaded.
Domestic service assets loaded.
Other assets loaded.
Post codes loaded.
Weather stations loaded.
Weather data loaded.


In [6]:
# Check data has loaded as intended

#df_notifs.head()
#df_ds_floc_master_data.head()
#df_else_floc_master_data.head()
#df_postcodes.head()
#df_stations.head()
#df_weather.head()

<a id='clean'></a>

## Clean data loaded into dataframes

Strip out all columns that will not be explicitly used by this particular analysis. These 'cleaning' algorithms will need to be tweaked for different analyses.

### Clean df_notifs dataframe
There are several code blocks below that build a dataframe for notifications analysis. Depending on the anlysis you want to do, select the appropriate code block.

In [7]:
#df_notifs.info()

#### Build Failure Analysis Dataframe
Use this code block to form dataframe for failure analysis

In [11]:
df = df_notifs

df.set_index('Notification')
clean = pd.DataFrame(index=df.index)

# Fields that can be directly copied to clean dataframe
make_copy = (
    ('Notification', 'notif'),
    ('Notif.date', 'notif_date')
    
)
for orig, new in make_copy:
    clean[new] = df[orig]

# Numeric columns
clean['notif'] = pd.to_numeric(df['Notification']*1., errors='coerce')# recast notification number from int64 to float64
clean['floc'] = pd.to_numeric(df['Functional Loc.'], errors='coerce')
clean['order'] = pd.to_numeric(df['Order'], errors='coerce')

# Categorical columns
# TODO: Maintain spreadsheet containing lists of all these codes and load them for application here
make_categorical = (  # has column, new_name, category pairs
    ('Job Type', 'job_type', None),
    ('ObjectPartCode', 'object_part_code', None),
    ('ObjPartCodeText', 'object_part_text', None),
    ('Damage Code', 'damage_code', None),
    ('Prob. code text', 'damage_code_text', None),
    ('Cause code', 'cause_code', None),
    ('Cause code text', 'cause_code_text', None),
    ('Unit', 'duration_units', None),
    ('Breakdown', 'breakdown', ('X','')),
    ('Object Code group', 'object_code_group', None),
    ('Obj.p. grp.txt.', 'object_group_text', None),
    ('Notifictn type', 'notif_type', ('SP','SF')),
    ('Cause Code group', 'cause_code_group', None),
    ('Cause grp. text', 'cause_group_text', None)
)

for column, new_name, cats in make_categorical:
    clean[new_name] = pd.Categorical(df[column], categories=cats)

clean_notifs = clean

#### Build Variable Volume Job Type Analysis Dataframe
Use this code block to form dataframe for VV Forecasting analysis

In [12]:
df = df_notifs

df.set_index('Notification')
clean = pd.DataFrame(index=df.index)

# Fields that can be directly copied to clean dataframe
make_copy = (
    ('Notification', 'notif'),
    ('Notif.date', 'notif_date')
    
)
for orig, new in make_copy:
    clean[new] = df[orig]

# Numeric columns
clean['notif'] = pd.to_numeric(df['Notification']*1., errors='coerce')# recast notification number from int64 to float64
clean['floc'] = pd.to_numeric(df['Functional Loc.'], errors='coerce')
clean['order'] = pd.to_numeric(df['Order'], errors='coerce')

# Categorical columns
clean['job_type'] = pd.Categorical(df['Job Type'], categories=None)

clean_notifs = clean

In [9]:
#clean_notifs.info()

### Clean df_ds_floc_master_data dataframe

This dataframe contains master data associated with all domestic gas distribution services in WA
Primary key is TPLNR field,(functional location).

In [10]:
#df_ds_floc_master_data.info()

In [13]:
df = df_ds_floc_master_data

df.set_index('TPLNR')
clean = pd.DataFrame(index=df.index)

# Fields that can be directly copied to clean dataframe
make_copy = (
    ('MTRMD_I', 'meter_model'),
    ('MTRDT_I', 'meter_install_date')
)
for orig, new in make_copy:
    clean[new] = df[orig]

# Numeric columns
clean['floc'] = pd.to_numeric(df['TPLNR'], errors='coerce')
clean['supply_pressure'] = pd.to_numeric(df['SUP_PRS'], errors='coerce')
clean['postcode'] = pd.to_numeric(df['PCODE'], errors='coerce')

# Categorical columns
make_categorical = (  # has column, new_name, category pairs
    ('RBNR', 'catalog_profile', None),
    ('LOCN', 'network', ('NM','SM','MA','BU','BS','KA','AL','GE'))
)

for column, new_name, cats in make_categorical:
    clean[new_name] = pd.Categorical(df[column], categories=cats)

clean_ds_floc_master_data = clean

In [12]:
#clean_ds_floc_master_data.info()

### Clean df_else_floc_master_data dataframe

This dataframe contains master data associated with all ATCO Gas assets other than domestic services loaded above
Primary key:TPLNR    #functional location.

In [13]:
#df_else_floc_master_data.info()

In [14]:
df = df_else_floc_master_data

df.set_index('Functional Loc.')
clean = pd.DataFrame(index=df.index)

# Numeric columns
clean['floc'] = pd.to_numeric(df['Functional Loc.'], errors='coerce')
clean['postcode'] = pd.to_numeric(df['Postal Code'], errors='coerce')

# Categorical columns
make_categorical = (  # has column, new_name, category pairs
    ('Catalog profile', 'catalog_profile', ('MAIN','GATESTN','REGSET','METERSET','VALVE','RECTFR','SERV_LINE','COMMETER')),
    ('Location', 'network', ('NM','SM','MA','BU','BS','KA','AL','GE')),
    ('City', 'suburb', None)
)

for column, new_name, cats in make_categorical:
    clean[new_name] = pd.Categorical(df[column], categories=cats)

clean_else_floc_master_data = clean

In [15]:
#clean_else_floc_master_data.info()

### Clean df_postcodes dataframe

This dataframe contains WA postcodes and their associated lat/lon.
Primary key: postcode

In [16]:
#df_postcodes.info()

In [15]:
df = df_postcodes

df.set_index('postcode')
clean = pd.DataFrame(index=df.index)

# Numeric columns
clean['postcode'] = pd.to_numeric(df['postcode'], errors='coerce')
clean['lat'] = pd.to_numeric(df['lat'], errors='coerce')
clean['lon'] = pd.to_numeric(df['long'], errors='coerce')

# Categorical column
clean['suburb'] = pd.Categorical(df['locality'], categories=None)

#clean = clean.set_index('postcode')
clean_postcodes = clean

In [18]:
#clean_postcodes.info()

### Clean df_stations dataframe

This dataframe contains a list of BOM weather stations in WA and their associated lat/lon.
Primary key: STN ID

In [19]:
#df_stations.info()

In [16]:
df = df_stations

df.set_index('STN ID')
clean = pd.DataFrame(index=df.index)

# Field that can be directly copied to clean dataframe
clean['station_id'] = df['STN ID']

# Numeric columns
clean['lat'] = pd.to_numeric(df['LAT'], errors='coerce')
clean['lon'] = pd.to_numeric(df['LON'], errors='coerce')

# Categorical columns
clean['station_name'] = pd.Categorical(df['NAME'], categories=None)

clean = clean.set_index('station_id')
clean_stations = clean

In [21]:
#clean_stations.info()

### Clean df_weather dataframe

This dataframe contains a list of features obtained from BOM weather stations in WA.
Primary key: None. >> Link with Station name and date

In [22]:
#df_weather.info()

In [24]:
#df = df_weather
df = all_stations
clean = pd.DataFrame()

# Datetime columns
clean['weather_date'] = pd.to_datetime(df['weather_date'])

# Numeric columns
clean['evapo_trans_0000_2400'] = pd.to_numeric(df['evapo_trans_0000_2400'], errors='coerce')
clean['rain_0900_0900'] = pd.to_numeric(df['rain_0900_0900'], errors='coerce')
clean['pan_evap_0900_0900'] = pd.to_numeric(df['pan_evap_0900_0900'], errors='coerce')
clean['max_temp'] = pd.to_numeric(df['max_temp'], errors='coerce')
clean['min_temp'] = pd.to_numeric(df['min_temp'], errors='coerce')
clean['max_rel_humidity'] = pd.to_numeric(df['max_rel_humidity'], errors='coerce')
clean['min_rel_humidity'] = pd.to_numeric(df['min_rel_humidity'], errors='coerce')
clean['avg_10m_wind_speed'] = pd.to_numeric(df['avg_10m_wind_speed'], errors='coerce')
clean['solar_radiation'] = pd.to_numeric(df['solar_radiation'], errors='coerce')

# Categorical columns
clean['station_name'] = pd.Categorical(df['station_name'], categories=None)

#clean = clean.set_index('station_name')
clean = clean.set_index('weather_date')

# Fill missing values
clean = clean.fillna(method='ffill')

clean_weather = clean

,evapo_trans_0000_2400,rain_0900_0900,pan_evap_0900_0900,max_temp,min_temp,max_rel_humidity,min_rel_humidity,avg_10m_wind_speed,solar_radiation,station_name
weather_date,,,,,,,,,,
2009-01-11,4.0,0.4,3.8,16.0,7.2,81.0,50.0,5.84,25.47,ALBANY AIRPORT
2009-02-11,4.1,0.0,4.0,15.5,10.2,81.0,51.0,4.34,26.80,ALBANY AIRPORT
2009-03-11,4.2,0.0,5.6,17.3,10.6,90.0,58.0,5.05,29.80,ALBANY AIRPORT
2009-04-11,4.5,0.0,5.4,21.4,9.4,89.0,58.0,6.31,26.36,ALBANY AIRPORT
2009-05-11,6.0,0.0,5.2,29.4,13.0,92.0,22.0,4.26,18.53,ALBANY AIRPORT
2009-06-11,5.3,1.0,3.2,29.7,16.4,95.0,37.0,4.06,17.84,ALBANY AIRPORT
2009-07-11,3.7,2.2,5.8,20.5,12.8,94.0,64.0,3.30,22.32,ALBANY AIRPORT
2009-08-11,5.3,0.2,4.2,23.0,11.9,97.0,43.0,3.87,30.52,ALBANY AIRPORT
2009-09-11,4.3,0.0,7.2,21.2,8.5,99.0,52.0,3.90,26.12,ALBANY AIRPORT


In [25]:
# Save concatenated files into a single file so above code does not need to continually be rerun
# Takes about one minute to run

output_filename = 'weather.xlsx'
sheet_name = 'Sheet1'
output = pd.ExcelWriter(path.join(data_folder, output_filename), engine='xlsxwriter')
clean_weather.to_excel(output, sheet_name)
output.save()

In [24]:
#clean_weather.info()

<a id='combine'></a>

## Combine dataframes into a single dataframe to be used for the analysis

### Combine asset master data.

In [26]:
# Join DS functional locations and other asset functional locations into a single dataframe
df = clean_ds_floc_master_data.append(clean_else_floc_master_data, sort=True)

df_assets = df.drop_duplicates(subset=['floc'])
df_assets = df_assets.set_index('floc')

#df_assets.info()

### Add asset data to maintenance data

In [27]:
# Join combined asset master data above to maintenance dataframe
df_maint = clean_notifs.merge(df_assets, left_on='floc', right_on='floc', how='left')

#df_maint = df_maint.set_index('notif')

df_maint.head()

,notif,notif_date,floc,order,job_type,catalog_profile,meter_install_date,meter_model,network,postcode,suburb,supply_pressure
0,300713965.0,2011-12-21,504107.0,NaN,SPH,MAIN,NaN,NaN,SM,6110.0,SOUTHERN RIVER,NaN
1,300719006.0,2011-12-21,673096.0,NaN,SPL,MAIN,NaN,NaN,SM,6111.0,KELMSCOTT,NaN
2,300719010.0,2011-12-21,810841.0,NaN,SPH,MAIN,NaN,NaN,NM,6019.0,WEMBLEY DOWNS,NaN
3,300719430.0,2011-12-21,27.0,NaN,SPH,MAIN,NaN,NaN,NM,6026.0,WOODVALE,NaN
4,300730505.0,2011-12-21,117547.0,NaN,SF1,DOMMETER,14.01.2014,M8A,SM,6153.0,NaN,1.25


### Combine Station dataframe with postcodes dataframe

In [28]:
# Calculate distances from each suburb centroid to idently closest weather station and then add station row to suburb dataframe
# Takes about three minutes to run

import numpy as np

result = pd.DataFrame()
clean_postcodes_stn = pd.DataFrame()

for postcode, pc_row in clean_postcodes.iterrows():
    min_dist = 1000
    for station_id, stn_row in clean_stations.iterrows():
        # Use Pythagoras, not accurate but I don't know how to make it better...
        dist = ((pc_row['lat']-stn_row['lat'])**2 + (pc_row['lon']-stn_row['lon'])**2)**0.5 
        if dist < min_dist:
            min_dist = dist
            p_code = pc_row['postcode']
            stn_name = stn_row['station_name']
        
        result = pd.DataFrame({'postcode': [p_code],
                               'station_name': [stn_name],
                               'distance': [min_dist]})
        
    result = result.reset_index(drop=True) 
    clean_postcodes_stn = clean_postcodes_stn.append(result, ignore_index=True)

clean_postcodes_stn.columns=['postcode', 'station_name', 'distance']   
clean_postcodes_stn = clean_postcodes_stn.drop_duplicates(subset=['postcode'])

#clean_postcodes_stn.info()

### Add weather station name to maintenance data

In [29]:
# Add station name to maintenance data
df_maint = df_maint.merge(clean_postcodes_stn, on='postcode', how='left')

df_maint.head()

,notif,notif_date,floc,order,job_type,catalog_profile,meter_install_date,meter_model,network,postcode,suburb,supply_pressure,station_name,distance
0,300713965.0,2011-12-21,504107.0,NaN,SPH,MAIN,NaN,NaN,SM,6110.0,SOUTHERN RIVER,NaN,JANDAKOT AERO,0.131159
1,300719006.0,2011-12-21,673096.0,NaN,SPL,MAIN,NaN,NaN,SM,6111.0,KELMSCOTT,NaN,BICKLEY,0.099496
2,300719010.0,2011-12-21,810841.0,NaN,SPH,MAIN,NaN,NaN,NM,6019.0,WEMBLEY DOWNS,NaN,SWANBOURNE,0.049014
3,300719430.0,2011-12-21,27.0,NaN,SPH,MAIN,NaN,NaN,NM,6026.0,WOODVALE,NaN,PERTH METRO,0.138307
4,300730505.0,2011-12-21,117547.0,NaN,SF1,DOMMETER,14.01.2014,M8A,SM,6153.0,NaN,1.25,JANDAKOT AERO,0.087419


### Add weather data to maintenance data

In [30]:
# Add weather to maintenance data
# Use weather station name and date as keys
df_complete = pd.merge(df_maint, clean_weather, left_on=['station_name','notif_date'], 
                       right_on=['station_name','weather_date'], how='left')

In [31]:
df_complete = df_complete.set_index(df_complete.columns[0])
df_complete.head()

,notif_date,floc,order,job_type,catalog_profile,meter_install_date,meter_model,network,postcode,suburb,...,distance,evapo_trans_0000_2400,rain_0900_0900,pan_evap_0900_0900,max_temp,min_temp,max_rel_humidity,min_rel_humidity,avg_10m_wind_speed,solar_radiation
notif,,,,,,,,,,,,,,,,,,,,,
300713965.0,2011-12-21,504107.0,NaN,SPH,MAIN,NaN,NaN,SM,6110.0,SOUTHERN RIVER,...,0.131159,7.3,0.0,10.8,28.9,13.6,97.0,32.0,3.87,34.47
300719006.0,2011-12-21,673096.0,NaN,SPL,MAIN,NaN,NaN,SM,6111.0,KELMSCOTT,...,0.099496,7.9,0.0,8.0,30.6,16.4,82.0,35.0,4.24,34.47
300719010.0,2011-12-21,810841.0,NaN,SPH,MAIN,NaN,NaN,NM,6019.0,WEMBLEY DOWNS,...,0.049014,6.4,0.0,36.8,25.8,16.9,91.0,61.0,4.63,34.24
300719430.0,2011-12-21,27.0,NaN,SPH,MAIN,NaN,NaN,NM,6026.0,WOODVALE,...,0.138307,7.2,0.0,2.4,29.2,17.4,89.0,38.0,3.37,34.38
300730505.0,2011-12-21,117547.0,NaN,SF1,DOMMETER,14.01.2014,M8A,SM,6153.0,NaN,...,0.087419,7.3,0.0,10.8,28.9,13.6,97.0,32.0,3.87,34.47


<a id='write'></a>

## Write cleaned and combined data to file

In [32]:
#csv_file = path.join(data_folder, 'corrective_maint_against_weather.csv')
csv_file = path.join(data_folder, 'corrective_maint_job_types.csv')
df_complete.to_csv(csv_file)

#csv_file = path.join(data_folder, 'binary_weather.csv')
#df_wide.to_csv(csv_file)

### Convert catagorical columns to binary to enable PCA dimensionality reduction later

In [9]:
df_wide = pd.DataFrame()

# Fields to directly copy to new analysis dataframe
make_copy = (
    ('weather_date', 'date'),
    #('evapo_trans_0000_2400', 'evapo_trans_0000_2400'),
    #('rain_0900_0900', 'rain_0900_0900'),
    #('pan_evap_0900_0900', 'pan_evap_0900_0900'),
    ('max_temp', 'max_temp'),
    ('min_temp', 'min_temp'),
    #('max_rel_humidity', 'max_rel_humidity'),
    #('min_rel_humidity', 'min_rel_humidity'),
    #('avg_10m_wind_speed', 'avg_10m_wind_speed'),
    #('solar_radiation', 'solar_radiation')
)
for orig, new in make_copy:
    df_wide[new] = df_complete[orig]

cat = pd.get_dummies(df_complete['catalog_profile'])
job = pd.get_dummies(df_complete['job_type'])
#net = pd.get_dummies(df_complete['network'])
#obj = pd.get_dummies(df_complete['object_part_code'])
#dam = pd.get_dummies(df_complete['damage_code'])
#cau = pd.get_dummies(df_complete['cause_code'])


df_wide = df_wide.join(cat)
df_wide = df_wide.join(job)
#df_wide = df_wide.join(net)
#df_wide = df_wide.append(obj, sort=False)
#df_wide = df_wide.append(dam, sort=False)
#df_wide = df_wide.append(cau, sort=False)


df_wide=df_wide.set_index('date')
df_wide=df_wide.interpolate()

In [10]:
df_wide.head()

,max_temp,min_temp,COMMETER,DOMMETER,GATESTN,MAIN,METERSET,RECTFR,REGSET,VALVE,...,SPN,SPO,SPP,SPR,SPT,SPU,SPW,SPX,SPY,SPZ
date,,,,,,,,,,,,,,,,,,,,,
2011-12-21,28.9,13.6,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-21,30.6,16.4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-21,25.8,16.9,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-21,29.2,17.4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-21,28.9,13.6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Write Analysis Frame to File

In [11]:
csv_file = path.join(data_folder, 'binary_weather.csv')
df_wide.to_csv(csv_file)